# Upstream Regulator Analysis Package Case Study

----------------------

Author: Mikayla Webster (m1webste@ucsd.edu)

Date: 11th January, 2018

----------------------

<a id='toc'></a>
## Table of Contents
1. [Import packages](#import)
2. [Background](#background)
3. [Transcription Factors](#tf)
4. [Background Network](#bn)
5. [HUVEC](#huvec)
    1. [Load DEG's](#loadhuvec)
    2. [Calculate p-values and z-scores](#pzhuvec)
    3. [Compare the Ingenuity Article's results to Ours](#comphuvec)
    4. [Display Our results](#displayhuvec)
6. [Breast Cancer](#brca)
    1. [Load DEG's](#loadbrca)
    2. [Calculate p-values and z-scores](#pzbrca)
    3. [Compare the Ingenuity Article's results to Ours](#compbrca)
    4. [Display Our results](#displaybrca)

## Background
<a id='background'></a>

This notebook attempts to validate our Upstream Regulator Analysis (URA) modules. Our modules are inspired by Ingenuity System's [Ingenuity Upstream Regulator Analysis in IPA®](http://pages.ingenuity.com/rs/ingenuity/images/0812%20upstream_regulator_analysis_whitepaper.pdf); this test case is inpired by Ingenuity System's corresponding paper, [Causal analysis approaches in Ingenuity Pathway Analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3928520/). This paper analyses two sets of Differencially Expressed Genes (DEG's): one from a breast cancer tumor and the other from Human Umbilical Vein Endothelial Cells (HUVEC). We run our version of URA on these same breast cancer and HUVEC DEG's, however using the [STRING database](https://string-db.org/) human protein interaction network as our background network. 

## Import packages
<a id='import'></a>

In [33]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import upstream regulator modules
import sys
code_path = '../ura'
sys.path.append(code_path)
import create_graph
import stat_analysis
reload(create_graph)
reload(stat_analysis)

# network visualization package
# (pip install visJS2jupyter)
import visJS2jupyter.visJS_module as visJS_module

## Transcription Factors
<a id='tf'></a>

Load our list of Transcription Factors (TF) we want to analyze. Gene symbols will be in all caps. 

In [2]:
TF_list = create_graph.create_TF_list(slowkow_bool=True,
                    slowkow_files=['../slowkow_databases/TRED_TF.txt',
                                   '../slowkow_databases/ITFP_TF.txt',
                                   '../slowkow_databases/ENCODE_TF.txt',
                                   '../slowkow_databases/Neph2012_TF.txt',
                                   '../slowkow_databases/TRRUST_TF.txt',
                                   '../slowkow_databases/Marbach2016_TF.txt'],
                    slowkow_sep = '\n',
                    jaspar_bool=True,
                    jaspar_file="../jaspar_genereg_matrix.txt")


TF_list = TF_list + ['TNF', 'IFNG', 'LBP'] # known regulators of interest missing from our TF databases

## Background Network
<a id='bn'></a>

Load our background network, available on the [STRING website](https://string-db.org/cgi/download.pl?UserId=9BGA8WkVMRl6&sessionId=HWUK6Dum9xC6&species_text=Homo+sapiens), and keep only our the information about our TF's. 

The function load_STRING_to_digraph can load any species' "protein actions" database from STRING. Just ensure your TF list and DEG list have the same naming convention as you background network (Homo Sapiens use all caps, Mus Musculus only capitalizes the first letter, etc.) Inconsistent naming can effect your results.

In [3]:
filename = "../9606.protein.actions.v10.5.txt"
confidence_filter=400
DG = create_graph.load_STRING_to_digraph(filename, confidence_filter, TF_list)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-7374...done.
Finished.
56 input query terms found dup hits:
	[(u'ENSP00000359550', 3), (u'ENSP00000447879', 2), (u'ENSP00000364076', 2), (u'ENSP00000348986', 2),
312 input query terms found no hit:
	[u'ENSP00000376684', u'ENSP00000289352', u'ENSP00000202788', u'ENSP00000373637', u'ENSP00000367802',
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


## HUVEC##
<a id='huvec'></a>

### Load HUVEC DEG's ###
<a id='loadhuvec'></a>

In [4]:
# DEG's
filename_huvec = '../geo2r_GSE2639_huvec.txt'

# TF's found to be statistically significant in Ingenuity Systems URA article
huvec_genes = ['TNF','IFNG','LBP', 'NFKB1', 'NFKB2', 'REL', 'RELA', 'RELB', 'PCBP3', 'PCBP2', 'PCBP1', 'PCBP4', 'NFKBIA']

# add DEG information to STRING background network
DEG_list_huvec, DEG_to_pvalue_huvec, DEG_to_updown_huvec = create_graph.create_DEG_list(filename_huvec, p_value_filter = 0.3)
DG_huvec = create_graph.add_updown_from_DEG(DG, DEG_to_updown_huvec) # will not overwrite original graph DG

### Calculate p-values and z-scores
<a id='pzhuvec'></a>

For a detailed explination of our p-value and z-score calculation functions, see our URA_Basic_Example notebook

In [5]:
# calculate p-values
p_values_huvec = stat_analysis.tf_pvalues(DG_huvec, DG, DEG_list_huvec)

# calculate z-scores
z_scores_huvec = stat_analysis.tf_zscore(DG_huvec, DEG_list_huvec, bias_filter = 0.25) # recommended bias filter is 0.25

### Compare the Ingenuity Article's results to Ours
<a id='comphuvec'></a>

These are the TF's found to be most relevant according to the Ingenuity Pathway Analysis paper. Optimally, these genes would rank very high. A rank of 0 is always best, while the rank associated with a z-score of zero is the worst. 

In this case, IFNG anf TNF are high ranking, therefore show coordination between our results and the Ingenuity article's results. LBP, PCBP1, PCBP2, PCBP4, and RELB have z-scores of zero, meaning not enough information exists about these genes to calculate a true z-score for them. PCBP3 has a NaN z-score because it is not in the graph.

In [6]:
stat_analysis.rank_and_score_df(z_scores_huvec, huvec_genes, remove_dups=True)

,rank,z-score
IFNG,0.0,3.549648
TNF,5.0,2.342606
NFKB2,12.0,1.732051
RELA,13.0,1.732051
NFKBIA,19.0,1.341641
REL,19.0,-1.341641
NFKB1,23.0,1.091089
LBP,34.0,0.000000
PCBP1,34.0,0.000000
PCBP2,34.0,0.000000


### Display Our results
<a id='displayhuvec'></a>

These are the most relevant TF's according to our analysis. Optimally, these should match the huvec genes defined above. However, using a different background network produces discrepancies.

IFNG, TNF, and TNFAIP3 show coordination between our results and the Ingenuity article's results.

In [7]:
top_act_huvec = stat_analysis.top_values(z_scores_huvec, act = True, abs_value = False, top = 10)
top_inh_huvec = stat_analysis.top_values(z_scores_huvec, act = False, abs_value = False, top = 10)
display(top_act_huvec.to_frame(name = 'activating z-score'))
display(top_inh_huvec.to_frame(name = 'inhibiting z-score'))

,activating z-score
IFNG,3.549648
CTF1,3.000000
STAT4,2.645751
STAT5A,2.529822
TNF,2.342606
HIF1A,2.236068
GATA3,2.236068
IRF3,2.236068
FOS,2.236068
SPG7,2.236068


,inhibiting z-score
PPARA,-2.449490
TNFAIP3,-2.000000
DDX41,-2.000000
CACNA2D4,-2.000000
CACNA2D2,-2.000000
GPSM1,-1.941451
SIRT1,-1.897367
STRAP,-1.732051
IRF8,-1.732051
NKX2-5,-1.732051


## Breast Cancer ###
<a id='brca'></a>

### Load Breast Cancer DEG's
<a id='loadbrca'></a>

In [8]:
filename_brca = '../geo2r_GSE11352_brca_48hours.txt'
brca_genes = ['ESR1', 'FSH', 'MEK', 'BRD4', 'MYC', 'MARK1', 'IL1B','NCOA3','PGR', 
              'EGR1', 'HIF1A', 'NR3C1','CTNNB1','TP53','SMARCE1','STAT5B']
DEG_list_brca, DEG_to_pvalue_brca, DEG_to_updown_brca = create_graph.create_DEG_list(filename_brca, p_value_filter = 0.05)
DG_brca = create_graph.add_updown_from_DEG(DG, DEG_to_updown_brca)

### Calculate p-values and z-scores
<a id='pzbrca'></a>

In [9]:
p_values_brca = stat_analysis.tf_pvalues(DG_brca, DG, DEG_list_brca)

z_scores_brca = stat_analysis.tf_zscore(DG_brca, DEG_list_brca, bias_filter = 0.25)

### Compare the Ingenuity Article's results to Ours
<a id='compbrca'></a>
TP53, MYC, and ESR1 are foud to be highly ranking, showing coordination between our results and the Ingenuity article's results.

In [10]:
stat_analysis.rank_and_score_df(z_scores_brca, brca_genes, act = True, abs_value = True, remove_dups=True)

,rank,z-score
TP53,0.0,-1.889822
MYC,2.0,1.666667
ESR1,3.0,1.414214
STAT5B,4.0,1.341641
NR3C1,5.0,1.000000
HIF1A,7.0,-0.577350
CTNNB1,8.0,0.447214
EGR1,8.0,-0.447214
BRD4,9.0,0.000000
NCOA3,9.0,0.000000


### Display Our results
<a id='displaybrca'></a>
MYCN, MYC, ESR1, and TP53 show coordination between our results and the Ingenuity article's results.

Four out of five of our top activating genes are associated with breast cancer according to literature. Our top three inhibiting genes are also associated with breast cancer according to literature. 

In [11]:
top_act_brca = stat_analysis.top_values(z_scores_brca, act = True, abs_value = False, top = 10)
top_inh_brca = stat_analysis.top_values(z_scores_brca, act = False, abs_value = False, top = 10)
display(top_act_brca.to_frame(name = 'activating z-score'))
display(top_inh_brca.to_frame(name = 'inhibiting z-score'))

,activating z-score
STAT1,1.732051
MYCN,1.732051
STAT3,1.732051
MYC,1.666667
SREBF1,1.414214
PHKB,1.414214
CCNA2,1.414214
ESR1,1.414214
SREBF2,1.414214
IQGAP1,1.414214


,inhibiting z-score
TP53,-1.889822
FOXO3,-1.414214
EGR2,-1.414214
NR1H3,-1.414214
FOXO4,-1.414214
AKT1,-1.414214
NR1H2,-1.000000
NFE2L2,-1.000000
TWIST1,-1.000000
FOXP3,-1.000000


In [22]:
X = nx.DiGraph([[1,2],[2,3],[3,4]])
d = {0:'A', 1:'B', 2:'C'}
nx.set_node_attributes(X, 'help', d)
x.nodes(data=True)

KeyError: 0

In [84]:
tf = 'MYCN'
node_list = DG.neighbors(tf) + [tf]
G = DG.subgraph(node_list) #get only top z-score node and neighbors
nodes = list(G.nodes())

DEG_filename = '../geo2r_GSE11352_brca_48hours.txt'
DEG_list, DEG_to_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, p_value_filter = 1) # get info for all
DEG_to_fld = {n: DEG_to_updown[n] for n in nodes if n in DEG_to_updown} # keep only those in the graph

nts = {n: -np.log(DEG_to_pvalue[n]) for n in nodes if n in DEG_to_updown} # keep only those in graph G
avg = np.average(list(nts.values()))
node_to_size = {n:nts[n] if n in nts else avg for n in nodes}
node_to_size, avg

MYC_in_edges = G.in_edges('MYCN')
G.remove_edges_from(MYC_in_edges)
G.in_edges('MYCN')

[]

In [104]:
def vis_tf_network(DG, tf, DEG_filename):

    # create sub graph
    node_list = DG.neighbors(tf) + [tf]
    G = DG.subgraph(node_list) #get only top z-score node and neighbors
    
    # remove arrows pointing to tf
    tf_in_edges = G.in_edges(tf)
    G.remove_edges_from(tf_in_edges)
    
    nodes = list(G.nodes())
    edges = list(G.edges()) 

    # define the initial positions of the nodes using networkx's spring_layout function
    pos = nx.spring_layout(G)
    
    # define node colors
    DEG_list, DEG_to_pvalue, DEG_to_updown = create_graph.create_DEG_list(DEG_filename, p_value_filter = 1) # get info for all
    node_to_fld = {n: DEG_to_updown[n] for n in nodes if n in DEG_to_updown} # keep only those in graph G
    nx.set_node_attributes(G, 'fold_change', 0) # give all nodes a default fold change of zero
    nx.set_node_attributes(G, 'fold_change', node_to_fld) # overwrite with actual fold change for those nodes who have one
    node_to_color = visJS_module.return_node_to_color(G,
                                                      field_to_map='fold_change',
                                                      cmap=plt.cm.bwr,
                                                      alpha = 1.0,
                                                      color_vals_transform = None,
                                                      ceil_val=10,
                                                      color_max_frac = 1.0,
                                                      color_min_frac = 0.0)
    
    node_to_color[tf] = 'yellow'
    
    # define node size
    nts = {n: (2+(-np.log(DEG_to_pvalue[n]))) for n in nodes if n in DEG_to_updown} # keep only those in graph G
    avg = np.average(list(nts.values()))
    node_to_size = {n:nts[n] if n in nts else avg for n in nodes}
    
    # define edge colors
    edge_to_color = {edge[0:2]:color[np.sign(edge[2]['sign'])] for edge in list(G.edges(data = True))}
    
    # define node attributes
    nodes_dict = [{"id":n,
                   "color": node_to_color[n],
                   "size_field": node_to_size[n],
                   "x":pos[n][0]*2200,
                   "y":pos[n][1]*2200} for n in nodes]
    node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

    # define edge attributes
    edges_dict = [{"source":node_map[edges[i][0]], 
                   "target":node_map[edges[i][1]], 
                   "color":edge_to_color[edges[i]]} for i in range(len(edges))]

    # display results
    return visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                            edge_arrow_to = True,
                                            node_size_multiplier = 10,
                                            node_size_transform = '',
                                            node_size_field = 'size_field',
                                            node_font_size = 35,
                                            node_font_color = 'black',
                                            edge_color_hover = 'orange',
                                            edge_color_highlight = 'orange',
                                            edge_width = 1.5,
                                            edge_hoverWidth = 5,
                                            edge_selection_width = 5,
                                            node_border_width_selected = 0,
                                            graph_id = 100)
    
vis_tf_network(DG_brca, 'MYCN', '../geo2r_GSE11352_brca_48hours.txt')  

In [15]:
gene1 = 'MYCN'
gene2 = 'FOXO3'

# extract sub graphs
node_list_1 = DG_brca.neighbors(gene1) + [gene1]
G_1 = DG_brca.subgraph(node_list_1) #get only top z-score node and neighbors
node_list_2 = DG_brca.neighbors(gene2) + [gene2]
G_2 = DG_brca.subgraph(node_list_2) #get only top z-score node and neighbors

# differenciate between these two graphs
G2_dict = {}
for n in node_list_2:
    G2_dict[n] = n + ' '  
nx.relabel_nodes(G_2, G2_dict, copy = False)

# combine two subgraphs into one
#G = nx.compose(G_1,G_2)
G = G_2
nodes = list(G.nodes())
edges = list(G.edges()) 

# define the initial positions of the nodes using networkx's spring_layout function
pos = nx.spring_layout(G)

# define node colors
color = {-1:'blue', 1:'red', 0: 'grey'} # red = act/up, blue = inh/down, no info = grey
node_to_color = {node[0]:color[np.sign(node[1]['updown'])] for node in list(G.nodes(data = True))}
node_to_color[gene1] = 'yellow'
node_to_color[gene2] = 'yellow'
node_to_color[gene1 + ' '] = 'yellow'
node_to_color[gene2 + ' '] = 'yellow'

# define edge colors
edge_to_color = {edge[0:2]:color[np.sign(edge[2]['sign'])] for edge in list(G.edges(data = True))}

# define node size transform
node_to_size = dict(zip(nodes, [3]*len(nodes)))
node_to_size[gene1] = 5
node_to_size[gene2] = 5
node_to_size[gene1 + ' '] = 5
node_to_size[gene2 + ' '] = 5

# define node and edge attributes
nodes_dict = [{"id":n,
               "color": node_to_color[n],
               "size_field": node_to_size[n],
               "x":pos[n][0]*2200,
               "y":pos[n][1]*2200} for n in nodes]

node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
               "color":edge_to_color[edges[i]]} for i in range(len(edges))]

# display results
visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                        edge_arrow_to = True,
                                        node_size_multiplier = 6,
                                        node_size_transform = '',
                                        node_size_field = 'size_field',
                                        node_font_size = 35,
                                        node_font_color = 'black',
                                        edge_color_opacity = 0.5,
                                        edge_color_hover = 'orange',
                                        edge_color_highlight = 'orange',
                                        edge_hoverWidth = 5,
                                        edge_selection_width = 5,
                                        node_border_width_selected = 0)

In [19]:
gene1 = 'MYCN'
gene2 = 'FOXO3'

# extract sub graphs
node_list_1 = DG_brca.neighbors(gene1) + [gene1]
G_1 = DG_brca.subgraph(node_list_1) #get only top z-score node and neighbors
node_list_2 = DG_brca.neighbors(gene2) + [gene2]
G_2 = DG_brca.subgraph(node_list_2) #get only top z-score node and neighbors

# differenciate between these two graphs
G2_dict = {}
for n in node_list_2:
    G2_dict[n] = n + ' '  
nx.relabel_nodes(G_2, G2_dict, copy = False)

# combine two subgraphs into one
#G = nx.compose(G_1,G_2)
G = G_1
nodes = list(G.nodes())
edges = list(G.edges()) 

# define the initial positions of the nodes using networkx's spring_layout function
pos = nx.spring_layout(G)

# define node colors
color = {-1:'blue', 1:'red', 0: 'grey'} # red = act/up, blue = inh/down, no info = grey
node_to_color = {node[0]:color[np.sign(node[1]['updown'])] for node in list(G.nodes(data = True))}
node_to_color[gene1] = 'yellow'
node_to_color[gene2] = 'yellow'
node_to_color[gene1 + ' '] = 'yellow'
node_to_color[gene2 + ' '] = 'yellow'

# define edge colors
edge_to_color = {edge[0:2]:color[np.sign(edge[2]['sign'])] for edge in list(G.edges(data = True))}

# define node size transform
node_to_size = dict(zip(nodes, [3]*len(nodes)))
node_to_size[gene1] = 5
node_to_size[gene2] = 5
node_to_size[gene1 + ' '] = 5
node_to_size[gene2 + ' '] = 5

# define node and edge attributes
nodes_dict = [{"id":n,
               "color": node_to_color[n],
               "size_field": node_to_size[n],
               "x":pos[n][0]*1200,
               "y":pos[n][1]*1200} for n in nodes]

node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
               "color":edge_to_color[edges[i]]} for i in range(len(edges))]

# display results
visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                        edge_arrow_to = False,
                                        node_size_multiplier = 6,
                                        node_size_transform = '',
                                        node_size_field = 'size_field',
                                        node_font_size = 25,
                                        node_font_color = 'black',
                                        edge_color_opacity = 0.5,
                                        edge_color_hover = 'orange',
                                        edge_color_highlight = 'orange',
                                        edge_hoverWidth = 3,
                                        edge_selection_width = 3,
                                        node_border_width_selected = 0,
                                        graph_id = 1)

In [18]:
gene1 = 'MYCN'
gene2 = 'FOXO3'

# extract sub graphs
node_list_1 = DG_brca.neighbors(gene1) + [gene1]
G_1 = DG_brca.subgraph(node_list_1) #get only top z-score node and neighbors
node_list_2 = DG_brca.neighbors(gene2) + [gene2]
G_2 = DG_brca.subgraph(node_list_2) #get only top z-score node and neighbors

# differenciate between these two graphs
G2_dict = {}
for n in node_list_2:
    G2_dict[n] = n + ' '  
nx.relabel_nodes(G_2, G2_dict, copy = False)

# combine two subgraphs into one
G = nx.compose(G_1,G_2)
nodes = list(G.nodes())
edges = list(G.edges()) 

# define the initial positions of the nodes using networkx's spring_layout function
pos = nx.spring_layout(G)

pos[gene1] = np.array([ 0.4,  0.7])
pos[gene2] = np.array([ 0.4,  0.55])
pos[gene1 + ' '] = np.array([ 0.4,  0.7])
pos[gene2 + ' '] = np.array([ 0.4,  0.55])

# define node colors
color = {-1:'blue', 1:'red', 0: 'grey'} # red = act/up, blue = inh/down, no info = grey
node_to_color = {node[0]:color[np.sign(node[1]['updown'])] for node in list(G.nodes(data = True))}
node_to_color[gene1] = 'yellow'
node_to_color[gene2] = 'yellow'
node_to_color[gene1 + ' '] = 'yellow'
node_to_color[gene2 + ' '] = 'yellow'

# define edge colors
edge_to_color = {edge[0:2]:color[np.sign(edge[2]['sign'])] for edge in list(G.edges(data = True))}

# define node size transform
#node_to_size = dict(zip(nodes, [3]*len(nodes)))
node_to_size = { n:3 if node_to_color[n] == 'grey' else 5 for n in nodes }

# define node and edge attributes
nodes_dict = [{"id":n,
               "color": node_to_color[n],
               "size_field": node_to_size[n],
               "x":pos[n][0]*2200,
               "y":pos[n][1]*2200} for n in nodes]

node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
               "color":edge_to_color[edges[i]]} for i in range(len(edges))]

# display results
visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                                        edge_arrow_to = True,
                                        node_size_multiplier = 6,
                                        node_size_transform = '',
                                        node_size_field = 'size_field',
                                        node_font_size = 35,
                                        node_font_color = 'black',
                                        edge_color_opacity = 0.1,
                                        edge_color_hover = 'orange',
                                        edge_color_highlight = 'orange',
                                        edge_hoverWidth = 5,
                                        edge_selection_width = 5,
                                        node_border_width_selected = 0,
                                        graph_id = 2)